In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def get_electon_results(url):
    response = requests.get(url)
    data = response.text
    
    soup = BeautifulSoup(data)
    table_div = soup.find('div', {'id': 'div1'})
    data_table = table_div.find_all('table')[0]
    table_rows = data_table.find_all('tr')
    
    constituency = table_rows[0].find_all('td')[0].text.strip()
    
    #convert to dataframe
    rows = []
    th = table_rows[2].find_all('th')
    header = [cell.text for cell  in th]
    
    for tr in table_rows[3:-1]:
        td = tr.find_all('td')
        row = [cell.text for cell in td]
        rows.append(row)
    
    df = pd.DataFrame(rows, columns = header)
    return constituency, df

In [3]:
df_results = None
seats = range(1,244) #total 243 seats
id2seat = {}
for i in seats:
    url = 'https://results.eci.gov.in/ACTRENDS2020/ConstituencywiseS04'+ str(i) +'.htm?ac=' + str(i)
    seat_name, df = get_electon_results(url)
    id2seat[i] = seat_name
    df.insert(0, 'seat', seat_name)
    df.insert(0, 'index', i) #'index' is seat number from 1 to 243. There are 243 seats in Bihar LA
    df_results = pd.concat([df_results, df])

In [4]:
df_results.shape

(3976, 9)

In [5]:
alliance = {
'NDA' : ['Bharatiya Janata Party', 'Janata Dal (United)', 'Vikassheel Insaan Party', 'Hindustani Awam Morcha (Secular)'],
'UPA' : ['Rashtriya Janata Dal', 'Indian National Congress', 'Communist Party of India  (Marxist-Leninist)  (Liberation)',
      'Communist Party of India  (Marxist)', 'Communist Party of India']
}

def get_alliance(party_name):
    group = 'Others'
    for k,v in alliance.items():
        if party_name in v:
            group = k
    return group
        
df_results['alliance'] = df_results['Party'].map(lambda x: get_alliance(x))

In [6]:
df_results.to_csv(r'D:\dev\workspace\bihar_election_2020\bihar_LA_election_2020.csv', index = False)

In [7]:
len(id2seat)

243

In [8]:
#two constituencies can have same name
seat2id = {}
for k,v in id2seat.items():
    if v in seat2id.keys():
        seat2id[v].append(k)
        print(v, seat2id[v])
    else:
        seat2id[v] = [k]

Bihar-Pipra [17, 42]
Bihar-Kalyanpur [16, 131]


In [9]:
df_results['seat'].nunique()

241

In [10]:
df_results['index'].nunique()

243

In [11]:
df_results.columns

Index(['index', 'seat', 'O.S.N.', 'Candidate', 'Party', 'EVM Votes',
       'Postal Votes', 'Total Votes', '% of Votes', 'alliance'],
      dtype='object')

In [12]:
int_cols = ['EVM Votes', 'Postal Votes', 'Total Votes']
for col in int_cols:
    df_results[col] = df_results[col].astype(int)

In [14]:
df_results.loc[df_results['seat'].str.contains('Kahalgaon'),:].sort_values(by = ['EVM Votes'], ascending = False)

,index,seat,O.S.N.,Candidate,Party,EVM Votes,Postal Votes,Total Votes,% of Votes,alliance
2,155,Bihar-Kahalgaon,3,PAWAN KUMAR YADAV,Bharatiya Janata Party,115326,212,115538,56.23,NDA
3,155,Bihar-Kahalgaon,4,SHUBHANAND MUKESH,Indian National Congress,72379,266,72645,35.36,UPA
12,155,Bihar-Kahalgaon,13,BIJAY KUMAR YADAV,Independent,3101,6,3107,1.51,Others
14,155,Bihar-Kahalgaon,15,NOTA,None of the Above,2188,4,2192,1.07,Others
0,155,Bihar-Kahalgaon,1,ANUJ KUMAR MANDAL,Nationalist Congress Party,2097,8,2105,1.02,Others
11,155,Bihar-Kahalgaon,12,MAHENDRA TANTI,Independent,1969,4,1973,0.96,Others
1,155,Bihar-Kahalgaon,2,KRISHNA KUMAR MANDAL,Bahujan Samaj Party,1346,4,1350,0.66,Others
4,155,Bihar-Kahalgaon,5,ANIL YADAV,Jan Adhikar Party (Loktantrik),1272,3,1275,0.62,Others
13,155,Bihar-Kahalgaon,14,SADANAND SINGH,Independent,1166,1,1167,0.57,Others
10,155,Bihar-Kahalgaon,11,BHOLA HARIJAN,Independent,854,5,859,0.42,Others


In [15]:
df_winner = df_results.groupby(['index']).apply(lambda x: x.nlargest(1, ['Total Votes'])).reset_index(drop = True)
df_winner['Party'].value_counts()

Rashtriya Janata Dal                                          75
Bharatiya Janata Party                                        74
Janata Dal (United)                                           43
Indian National Congress                                      19
Communist Party of India  (Marxist-Leninist)  (Liberation)    12
All India Majlis-E-Ittehadul Muslimeen                         5
Hindustani Awam Morcha (Secular)                               4
Vikassheel Insaan Party                                        4
Communist Party of India  (Marxist)                            2
Communist Party of India                                       2
Lok Jan Shakti Party                                           1
Independent                                                    1
Bahujan Samaj Party                                            1
Name: Party, dtype: int64

In [16]:
df_winner['alliance'].value_counts()

NDA       125
UPA       110
Others      8
Name: alliance, dtype: int64

In [17]:
df_top2 = df_results.groupby(['index']).apply(lambda x: x.nlargest(2, ['Total Votes'])).reset_index(drop = True)

In [18]:
df_top2.head(10)

,index,seat,O.S.N.,Candidate,Party,EVM Votes,Postal Votes,Total Votes,% of Votes,alliance
0,1,Bihar-Valmikinagar,1,Dhirendra Pratap Singh alias Rinku singh,Janata Dal (United),74777,129,74906,38.32,NDA
1,1,Bihar-Valmikinagar,4,Rajesh Singh,Indian National Congress,52952,369,53321,27.28,UPA
2,2,Bihar-Ramnagar,1,Bhagirathi Devi,Bharatiya Janata Party,75360,63,75423,39.57,NDA
3,2,Bihar-Ramnagar,2,Rajesh Ram,Indian National Congress,59273,354,59627,31.28,UPA
4,3,Bihar-Narkatiaganj,4,RASHMI VARMA,Bharatiya Janata Party,75345,139,75484,45.85,NDA
5,3,Bihar-Narkatiaganj,5,VINAY VARMA,Indian National Congress,53826,524,54350,33.02,UPA
6,4,Bihar-Bagaha,2,RAM SINGH,Bharatiya Janata Party,89882,131,90013,49.51,NDA
7,4,Bihar-Bagaha,1,JAYESH MANGLAM SINGH,Indian National Congress,59388,605,59993,33,UPA
8,5,Bihar-Lauriya,2,Vinay Bihari,Bharatiya Janata Party,77852,75,77927,49.48,NDA
9,5,Bihar-Lauriya,3,Shambhu Tiwari,Rashtriya Janata Dal,48680,243,48923,31.06,UPA


In [25]:
low_margin = 1000
vote_diff_between_top2 = df_top2.groupby('index').apply(lambda x: x['Total Votes'].iloc[0] - x['Total Votes'].iloc[1])
#df_top2.loc[vote_diff_between_top2<low_margin,:]
seat_nums_with_low_margin = vote_diff_between_top2[vote_diff_between_top2<low_margin].index
print('num seats with low margin:', len(seat_nums_with_low_margin))

num seats with low margin: 11


In [24]:
df_low_margin = df_top2.loc[df_top2['index'].isin(seat_nums_with_low_margin),:]
df_low_margin.shape

(22, 10)

In [21]:
df_low_margin.groupby('index').apply(lambda x: x['Party'].iloc[0]).value_counts()

Janata Dal (United)         4
Rashtriya Janata Dal        3
Communist Party of India    1
Lok Jan Shakti Party        1
Independent                 1
Bharatiya Janata Party      1
dtype: int64

In [30]:
def get_seat_numbers(df, party_name):
    return df.loc[df['Party']==party_name,'index'].values
    

In [41]:
ljp_contested_seats = get_seat_numbers(df_results, 'Lok Jan Shakti Party')
jdu_contested_seats = get_seat_numbers(df_results, 'Janata Dal (United)')
bjp_contested_seats = get_seat_numbers(df_results, 'Bharatiya Janata Party')
ljp_jdu_common_seats = set(jdu_contested_seats).intersection(set(ljp_contested_seats))
ljp_bjp_common_seats = set(bjp_contested_seats).intersection(set(ljp_contested_seats))
jdu_bjp_common_seats = set(bjp_contested_seats).intersection(set(jdu_contested_seats))
print('JDU contested sets:', len(jdu_contested_seats))
print('LJP contested sets:', len(ljp_contested_seats))
print('Seats with LJP and JDU:',len(ljp_jdu_common_seats))
print('Seats with LJP and BJP:',len(ljp_bjp_common_seats))
print('Seats with JDU and BJP:',len(jdu_bjp_common_seats))

JDU contested sets: 115
LJP contested sets: 135
Seats with LJP and JDU: 113
Seats with LJP and BJP: 6
Seats with JDU and BJP: 0


In [35]:
df_results['Party'].unique()

array(['Janata Dal (United)', 'Bahujan Samaj Party',
       'Lok Jan Shakti Party', 'Indian National Congress',
       'The Plurals Party', 'Bhartiya Party (Loktantrik)',
       'Jan Sangharsh Dal,', 'Jan Adhikar Party (Loktantrik)',
       'Independent', 'None of the Above', 'Bharatiya Janata Party',
       'Rashtriya Lok Samta Party', 'Fauji Kisan Party',
       'Janta Dal Rashtravadi', 'Bhartiya Panchyat Party',
       'Nationalist Congress Party', 'Bhartiya Sablog Party',
       'Bharat Jan Jagran Dal', 'Jai Maha Bharath Party',
       'Lok Shakti Party (Loktantrik)', 'Rashtriya Jansambhavna Party',
       'Bahujan Maha Party', 'Bihar Lok Nirman Dal',
       'Bhartiya Rashtriya Dal', 'Janvadi Party(Socialist)',
       'Rashtriya Janata Dal', 'Jagrook Janta Party',
       'Aazad Samaj Party (Kanshi Ram)', 'Loktantrik Jan Swaraj Party',
       'Vanchit Samaj Party', 'Proutist Sarva Samaj', 'Apna Kisan Party',
       'Rashtriya mahan Gantantra Party', 'Janata Congress',
       'People